**CNN**

In [8]:
import numpy as np
from keras.datasets import mnist
import tensorflow as tf

# Tải và chuẩn bị dữ liệu
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train_scaled = np.array(X_train)/255.
X_test_scaled = np.array(X_test)/255.

# Chuyển đổi nhãn thành one-hot encoding
encoder = OneHotEncoder()
encoder.fit(y_train.reshape(-1,1))
y_train = encoder.transform(y_train.reshape(-1,1)).toarray()
y_test = encoder.transform(y_test.reshape(-1,1)).toarray()

# Xây dựng mô hình CNN
inp = Input(shape=(28, 28, 1))
cnn = Conv2D(filters=8, kernel_size=3, activation='relu')(inp)
pooling = MaxPooling2D(pool_size=(2, 2))(cnn)
drop = Dropout(0.2)(pooling)

cnn = Conv2D(filters=16, kernel_size=4, activation='relu')(drop)
pooling = MaxPooling2D(pool_size=(2, 2))(cnn)
drop = Dropout(0.2)(pooling)

cnn = Conv2D(filters=32, kernel_size=4, activation='relu')(drop)
pooling = MaxPooling2D(pool_size=(2, 2))(cnn)

f = Flatten()(pooling)
fc1 = Dense(units=32, activation='relu')(f)
fc2 = Dense(units=16, activation='relu')(fc1)
out = Dense(units=10, activation='softmax')(fc2)

model = Model(inputs=inp, outputs=out)

# Biên dịch mô hình
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train_scaled, y_train, batch_size=64, epochs=10, validation_data=(X_test_scaled, y_test))

# Lưu mô hình vào tệp .h5
model.save('deep_leaning_model.h5')


Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.6648 - loss: 0.9863 - val_accuracy: 0.9572 - val_loss: 0.1448
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9405 - loss: 0.1930 - val_accuracy: 0.9702 - val_loss: 0.0947
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9587 - loss: 0.1360 - val_accuracy: 0.9783 - val_loss: 0.0723
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9670 - loss: 0.1081 - val_accuracy: 0.9816 - val_loss: 0.0568
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9722 - loss: 0.0906 - val_accuracy: 0.9821 - val_loss: 0.0579
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9749 - loss: 0.0813 - val_accuracy: 0.9847 - val_loss: 0.0488
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9757 - loss: 0.0759 - val_accuracy: 0.9879 - val_loss: 0.0400
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9790 - loss: 0.0690 - val_a

**Logistic regression**

In [3]:
import numpy as np
import joblib
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder

# Tải dữ liệu MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Chuẩn hóa dữ liệu
X_train_scaled = np.array([x.ravel() / 255.0 for x in X_train])
X_test_scaled = np.array([x.ravel() / 255.0 for x in X_test])

# Thêm bias vào dữ liệu đầu vào
X_train_scaled = np.hstack((np.ones((X_train_scaled.shape[0], 1)), X_train_scaled))  # Thêm cột bias (1, 784)
X_test_scaled = np.hstack((np.ones((X_test_scaled.shape[0], 1)), X_test_scaled))    # Thêm cột bias (1, 784)

# One-Hot Encoding cho nhãn
encoder = OneHotEncoder()
encoder.fit(y_train.reshape(-1, 1))
y_train = encoder.transform(y_train.reshape(-1, 1)).toarray()
y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()

# Hàm dự đoán
def predict(X, w):
    h = np.dot(X, w)
    softmax = np.exp(h - np.max(h, axis=1, keepdims=True))  # Cách tính softmax ổn định hơn
    y_pred = softmax / np.sum(softmax, axis=1, keepdims=True)
    return y_pred

# Hàm tính loss
def loss(X, w, y):
    y_pred = predict(X, w)
    return -np.sum(y * np.log(y_pred + 1e-15)) / y.shape[0]  # Thêm epsilon để tránh log(0)

# Hàm tính accuracy
def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_true, axis=1))

# Hàm tính gradient
def gradient(X, w, y):
    y_pred = predict(X, w)
    delta = y_pred - y
    return np.dot(X.T, delta) / X.shape[0]

# Hàm huấn luyện
def train(X, y, w, lr, n_epoch):
    for epoch in range(n_epoch):
        w -= lr * gradient(X, w, y)
        current_loss = loss(X, w, y)
        
        y_pred = predict(X, w)  # Dự đoán để tính độ chính xác
        current_accuracy = accuracy(y, y_pred)
        print(f'Epoch {epoch}, Loss: {current_loss:.4f}, Accuracy: {current_accuracy * 100:.2f}%')
    return w

# Thiết lập tham số huấn luyện
lr = 0.1  # Tốc độ học
n_epoch = 50  # Số epoch
w = np.zeros((X_train_scaled.shape[1], 10))  # Trọng số khởi tạo (bao gồm cả bias)

# Huấn luyện mô hình
w = train(X_train_scaled, y_train, w, lr, n_epoch)


# Lưu mô hình vào tệp .h5
with h5py.File('logistic_regression_model.h5', 'w') as f:
    f.create_dataset('weights', data=w)

print("Mô hình đã được lưu thành công.")


Epoch 0, Loss: 2.1969, Accuracy: 67.24%
Epoch 1, Loss: 2.1006, Accuracy: 70.69%
Epoch 2, Loss: 2.0115, Accuracy: 72.72%
Epoch 3, Loss: 1.9291, Accuracy: 73.91%
Epoch 4, Loss: 1.8526, Accuracy: 74.81%
Epoch 5, Loss: 1.7817, Accuracy: 75.49%
Epoch 6, Loss: 1.7160, Accuracy: 76.06%
Epoch 7, Loss: 1.6551, Accuracy: 76.66%
Epoch 8, Loss: 1.5986, Accuracy: 77.19%
Epoch 9, Loss: 1.5463, Accuracy: 77.67%
Epoch 10, Loss: 1.4978, Accuracy: 78.10%
Epoch 11, Loss: 1.4527, Accuracy: 78.47%
Epoch 12, Loss: 1.4108, Accuracy: 78.88%
Epoch 13, Loss: 1.3718, Accuracy: 79.22%
Epoch 14, Loss: 1.3354, Accuracy: 79.50%
Epoch 15, Loss: 1.3015, Accuracy: 79.74%
Epoch 16, Loss: 1.2698, Accuracy: 79.99%
Epoch 17, Loss: 1.2401, Accuracy: 80.27%
Epoch 18, Loss: 1.2123, Accuracy: 80.50%
Epoch 19, Loss: 1.1861, Accuracy: 80.72%
Epoch 20, Loss: 1.1616, Accuracy: 80.97%
Epoch 21, Loss: 1.1385, Accuracy: 81.21%
Epoch 22, Loss: 1.1167, Accuracy: 81.38%
Epoch 23, Loss: 1.0961, Accuracy: 81.53%
Epoch 24, Loss: 1.0766, Ac

**App**

In [15]:
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
from PIL import Image, ImageOps, ImageDraw
import tensorflow as tf
import h5py
import cv2  # OpenCV for additional preprocessing
from tensorflow.keras.models import load_model

# Load CNN model
model_cnn = load_model('deep_learning_model.h5')

# Load logistic regression weights
with h5py.File('logistic_regression_model.h5', 'r') as f:
    weights = f['weights'][:]

# Build logistic regression model
model_logistic = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=10, activation='softmax', input_shape=(785,))
])

# Preprocess image, detect contours, and segment individual digits
def preprocess_and_segment(image):
    # Convert to grayscale and resize to (28, 28)
    image = np.array(image.convert('L'))
    
    # Apply binary threshold
    _, thresholded = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Apply dilation to make digits more distinct
    kernel = np.ones((3, 3), np.uint8)  # Define the kernel for dilation
    dilated = cv2.dilate(thresholded, kernel, iterations=1)  # Apply dilation
    
    # Find contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort contours from left to right
    contours = sorted(contours, key=lambda x: cv2.boundingRect(x)[0])

    # Extract each digit using bounding boxes around contours
    digit_images = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 5 and h > 5:  # Filter out small contours that are not digits
            digit = dilated[y:y+h, x:x+w]
            
            # Resize each digit to 28x28
            digit_resized = cv2.resize(digit, (28, 28), interpolation=cv2.INTER_AREA)
            digit_images.append(Image.fromarray(digit_resized))

    return digit_images

# Prediction function for each segmented digit
def predict_digits(digit_images):
    predictions_cnn = []
    predictions_logistic = []
    
    for digit in digit_images:
        # Convert digit image to numpy array, flatten, and normalize
        img_array = np.array(digit).reshape(1, -1) / 255.0  # Shape should be (1, 784)

        # Predict with CNN (reshape to 4D)
        pred_cnn = model_cnn.predict(img_array.reshape(-1, 28, 28, 1))
        
        # Add bias for logistic regression (1, 785)
        img_array_with_bias = np.hstack([np.ones((1, 1)), img_array])

        # Predict with logistic regression
        pred_logistic = model_logistic.predict(img_array_with_bias)

        # Append results
        predictions_cnn.append(np.argmax(pred_cnn))
        predictions_logistic.append(np.argmax(pred_logistic))
    
    return predictions_cnn, predictions_logistic

# Load and predict image from file
def load_and_predict():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        try:
            image = Image.open(file_path)
            digit_images = preprocess_and_segment(image)
            if digit_images:
                labels_cnn, labels_logistic = predict_digits(digit_images)
                result_label.config(text=f"CNN Predictions: {labels_cnn}\nLogistic Predictions: {labels_logistic}")
            else:
                result_label.config(text="No digits found.")
        except Exception as e:
            messagebox.showerror("Error", f"Invalid image format.\n{e}")

# Reset canvas and results
def reset_canvas():
    canvas.delete("all")
    result_label.config(text="")

# Draw on canvas
def paint(event):
    x1, y1, x2, y2 = (event.x - 2), (event.y - 2), (event.x + 2), (event.y + 2)
    canvas.create_oval(x1, y1, x2, y2, fill="black", width=5)
    draw.line([x1, y1, x2, y2], fill="black", width=5)

# Predict from drawn input on canvas
def predict_drawing():
    image = ImageOps.invert(image_draw).resize((200, 200))  # Resize to standard canvas size
    digit_images = preprocess_and_segment(image)  # Segment digits
    
    if digit_images:
        labels_cnn, labels_logistic = predict_digits(digit_images)
        result_label.config(text=f"CNN Predictions: {labels_cnn}\nLogistic Predictions: {labels_logistic}")
    else:
        result_label.config(text="No digits found.")

# GUI setup
root = tk.Tk()
root.title("Handwriting Recognition App")

load_button = tk.Button(root, text="Upload Image", command=load_and_predict)
load_button.pack(pady=5)

canvas = tk.Canvas(root, width=200, height=200, bg="white")
canvas.pack(pady=10)
canvas.bind("<B1-Motion>", paint)

image_draw = Image.new("L", (200, 200), "white")
draw = ImageDraw.Draw(image_draw)

predict_button = tk.Button(root, text="Predict Drawing", command=predict_drawing)
predict_button.pack(pady=5)

reset_button = tk.Button(root, text="Reset", command=reset_canvas)
reset_button.pack(pady=5)

result_label = tk.Label(root, text="", font=("Arial", 12))
result_label.pack(pady=10)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
